In [2]:
from google.colab import files

uploaded = files.upload()

Saving training_set.zip to training_set.zip
Saving training_set_pixel_size_and_HC.csv to training_set_pixel_size_and_HC (1).csv


In [88]:
import zipfile

with zipfile.ZipFile("training_set.zip", "r") as zip_ref:
    zip_ref.extractall("training_set")

images = "training_set/training_set"

In [89]:
ultrasound_images = [
    f for f in os.listdir(images)
    if f.endswith(".png") and "Annotation" not in f
]

In [90]:
import pandas as pd

labels = pd.read_csv("training_set_pixel_size_and_HC.csv")
labels.head()

,filename,pixel size(mm),head circumference (mm)
0,000_HC.png,0.069136,44.30
1,001_HC.png,0.089659,56.81
2,002_HC.png,0.062033,68.75
3,003_HC.png,0.091291,69.00
4,004_HC.png,0.061240,59.81


In [91]:
print("Number of ultrasound images:", len(ultrasound_images))
print("Number of labels:", len(labels))


Number of ultrasound images: 999
Number of labels: 999


In [92]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

labels = labels["head circumference (mm)"].values

X_train_files, X_test_files, y_train, y_test = train_test_split(
    ultrasound_images,
    labels,
    test_size=0.2,
    random_state=42
)

In [96]:
import cv2
import numpy as np

def load_images(folder):
    X = []
    for i in folder:
        path = os.path.join(images, i)
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (400, 270))
        X.append(image)
    return np.array(X)

X_train = load_images(X_train_files)
X_test  = load_images(X_test_files)

X_train = X_train[..., np.newaxis]
X_test   = X_test[..., np.newaxis]

In [99]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(270, 400, 1)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mae',
    metrics=['mae']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 268, 398, 8)    │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 134, 199, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 132, 197, 16)   │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 66, 98, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 103488)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │     3,311,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,312,929 (12.64 MB)

 Trainable params: 3,312,929 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [98]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=16
)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 50s 945ms/step - loss: 109.2436 - mae: 109.2436 - val_loss: 59.6564 - val_mae: 59.6564
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 43s 853ms/step - loss: 55.2651 - mae: 55.2651 - val_loss: 56.2596 - val_mae: 56.2596
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 46s 916ms/step - loss: 49.8375 - mae: 49.8375 - val_loss: 55.9235 - val_mae: 55.9235
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 43s 852ms/step - loss: 40.1503 - mae: 40.1503 - val_loss: 57.2267 - val_mae: 57.2267
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 44s 883ms/step - loss: 34.8159 - mae: 34.8159 - val_loss: 57.4162 - val_mae: 57.4162
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 45s 896ms/step - loss: 29.8064 - mae: 29.8064 - val_loss: 60.2497 - val_mae: 60.2497
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 80s 855ms/step - loss: 26.8230 - mae: 26.8230 - val_loss: 59.9715 - val_mae: 59.9715
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 45s 897ms/step - loss: 23.0296 - mae: 23.0296 - val_loss: 60.2097 - val_mae: 60.2097
Epoch 9/10
50/

In [71]:
val_loss, val_mae = model.evaluate(X_test, y_test)
print("Validation MAE:", val_mae, "mm")

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 694ms/step - loss: 31.6767 - mae: 31.6767
Validation MAE: 30.25931167602539 mm
